In [1]:
from ludwig.api import LudwigModel
import os
import yaml
import logging

PyTorch version 2.1.0+cu118 available.


/home/ray/anaconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["HUGGING_FACE_HUB_TOKEN"] = os.getenv("HUGGING_FACE_API_KEY")

In [4]:
#Ludwig conf
conf_str = """
model_type: llm
base_model: mistralai/Mistral-7B-v0.1
# base_model: alexsherstinsky/Mistral-7B-v0.1-sharded
# base_model: Siddharthvij10/MistralSharded2
quantization:
  bits: 4
adapter:
  type: lora
prompt:
  template: |
    {instruction}
    {input}
input_features:
  - name: prompt
    type: text
    preprocessing:
      max_sequence_length: 256
output_features:
  - name: output
    type: text
    preprocessing:
      max_sequence_length: 256
trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01
preprocessing:
  sample_ratio: 0.1
"""

conf = yaml.safe_load(conf_str)

In [5]:
#Training...
model = LudwigModel(config=conf, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
model.save("results")

Setting generation max_new_tokens to 256 to correspond with the max sequence length assigned to the output feature or the global max sequence length. This will ensure that the correct number of tokens are generated at inference time. To override this behavior, set `generation.max_new_tokens` to a different value in your Ludwig config.

╒════════════════════════╕
│ EXPERIMENT DESCRIPTION │
╘════════════════════════╛

╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                              │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                         │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Out

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:57<00:00, 28.67s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


Done.
No padding token id found. Using eos_token as pad_token.
Loaded HuggingFace implementation of mistralai/Mistral-7B-v0.1 tokenizer
No padding token id found. Using eos_token as pad_token.
Trainable Parameter Summary For Fine-Tuning
Fine-tuning with adapter: lora
trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.04703666202518836

╒══════════╕
│ TRAINING │
╘══════════╛

Creating fresh model training run.
Training for 10920 step(s), approximately 3 epoch(s).
Early stopping policy: 5 round(s) of evaluation, or 18200 step(s), approximately 5 epoch(s).

Starting with step 0, epoch: 0
Training:  33%|███████████████████████                                              | 3640/10920 [2:28:45<9:19:05,  4.61s/it, loss=0.0696]
Running evaluation for step: 3640, epoch: 1
Evaluation valid: 100%|█████████████████████████████████████████████████████████████████████████████████| 260/260 [21:13<00:00,  4.90s/it]
Input: Classify the type of the product listed in the following